In [1]:
import datetime
import glob
import hashlib
import re

import numpy as np
import pandas as pd
import sklearn.metrics.pairwise
import sklearn.neighbors
import sklearn.preprocessing
import sqlparse

from typing import Dict

In [58]:
pgfiles = glob.glob('data/extracted/simple/postgresql*.csv')
display(pgfiles)

['data/extracted/simple/postgresql-2021-12-06_160118.csv',
 'data/extracted/simple/postgresql-2021-12-06_160210.csv',
 'data/extracted/simple/postgresql-2021-12-06_160202.csv',
 'data/extracted/simple/postgresql-2021-12-06_160154.csv',
 'data/extracted/simple/postgresql-2021-12-06_160132.csv',
 'data/extracted/simple/postgresql-2021-12-06_160149.csv',
 'data/extracted/simple/postgresql-2021-12-06_160207.csv',
 'data/extracted/simple/postgresql-2021-12-06_160205.csv',
 'data/extracted/simple/postgresql-2021-12-06_160121.csv',
 'data/extracted/simple/postgresql-2021-12-06_160048.csv',
 'data/extracted/simple/postgresql-2021-12-06_160138.csv',
 'data/extracted/simple/postgresql-2021-12-06_160127.csv',
 'data/extracted/simple/postgresql-2021-12-06_160146.csv',
 'data/extracted/simple/postgresql-2021-12-06_160135.csv',
 'data/extracted/simple/postgresql-2021-12-06_160129.csv',
 'data/extracted/simple/postgresql-2021-12-06_160124.csv',
 'data/extracted/simple/postgresql-2021-12-06_160151.csv

In [59]:
# https://www.postgresql.org/docs/13/runtime-config-logging.html#RUNTIME-CONFIG-LOGGING-CSVLOG
PG_LOG_COLUMNS = [
    'log_time',
    'user_name',
    'database_name',
    'process_id',
    'connection_from',
    'session_id',
    'session_line_num',
    'command_tag',
    'session_start_time',
    'virtual_transaction_id',
    'transaction_id',
    'error_severity',
    'sql_state_code',
    'message',
    'detail',
    'hint',
    'internal_query',
    'internal_query_pos',
    'context',
    'query',
    'query_pos',
    'location',
    'application_name',
    'backend_type',
]


df = pd.concat(
    pd.read_csv(pgfile,
                names=PG_LOG_COLUMNS,
                parse_dates=['log_time', 'session_start_time'],
                usecols=['log_time', 'session_start_time', 'command_tag', 'message', 'detail'],
                header=None,
                index_col=False)
    for pgfile in pgfiles
)
print(df.shape)
print(df.columns)
print(set(df['command_tag']))

(603434, 5)
Index(['log_time', 'command_tag', 'session_start_time', 'message', 'detail'], dtype='object')
{nan, 'DELETE', 'BEGIN', 'INSERT', 'UPDATE', 'SET', 'SHOW', 'SELECT', 'COMMIT', 'ROLLBACK'}


## Extracting the relevant queries.

In [60]:
commands = ['SELECT', 'INSERT', 'UPDATE', 'DELETE']

def extract_query(message):
    for command in commands:
        idx = message.find(command)
        if idx != -1:
            query = message[idx:]
            return query
    return ''

df['query'] = df['message'].apply(extract_query)
df['query'].iloc[0]

'UPDATE stock   SET S_QUANTITY = 38 ,        S_YTD = S_YTD + 1,        S_ORDER_CNT = S_ORDER_CNT + 1,        S_REMOTE_CNT = S_REMOTE_CNT + 0  WHERE S_I_ID = 70631    AND S_W_ID = 1'

In [61]:
def extract_parameters_detail(detail):
    detail = str(detail)
    prefix = 'parameters: '
    idx = detail.find(prefix)
    if idx != -1:
        parameter_list = detail[idx + len(prefix):]
        params = {}
        for pstr in parameter_list.split(', '):
            pnum, pval = pstr.split(' = ')
            assert pnum.startswith('$')
            assert pnum[-1].isdigit()
            params[pnum] = pval
        return params
    return {}

df['params_detail'] = df['detail'].apply(extract_parameters_detail)
df['params_detail']

0        {}
1        {}
2        {}
3        {}
4        {}
         ..
28203    {}
28204    {}
28205    {}
28206    {}
28207    {}
Name: params_detail, Length: 603434, dtype: object

In [62]:
def substitute_parameters(query, parameters):
    for k, v in parameters.items():
        query = query.replace(k, v)
    return query

df['query_simple'] = df.apply(lambda row: substitute_parameters(row['query'], row['params_detail']), axis=1)
df['query_simple']

0        UPDATE stock   SET S_QUANTITY = 38 ,        S_...
1        UPDATE stock   SET S_QUANTITY = 76 ,        S_...
2        UPDATE stock   SET S_QUANTITY = 43 ,        S_...
3        UPDATE stock   SET S_QUANTITY = 73 ,        S_...
4        UPDATE stock   SET S_QUANTITY = 89 ,        S_...
                               ...                        
28203    INSERT INTO order_line (OL_O_ID, OL_D_ID, OL_W...
28204    INSERT INTO order_line (OL_O_ID, OL_D_ID, OL_W...
28205    UPDATE stock   SET S_QUANTITY = 89 ,        S_...
28206    UPDATE stock   SET S_QUANTITY = 43 ,        S_...
28207    UPDATE stock   SET S_QUANTITY = 47 ,        S_...
Name: query_simple, Length: 603434, dtype: object

## Anonymizer: salt and hash non-date non-digit strings.

In [5]:
ANONYMIZE = False

SALT = 'andycannotsay.com'.encode('utf-8')
DATE_REGEX = re.compile(r'\d{4}-\d{2}-\d{2}.*')
DIGITS_REGEX = re.compile(r'\d+\.?\d*')

def anonymize(sql):
    cleaned_tokens = []

    # TODO(WAN): sqlparse.parse is actually quite slow.
    # Do we really need this?
    parsed = sqlparse.parse(sql)
    if len(parsed) == 0:
        return ''
    
    assert len(parsed) == 1
    tokens = parsed[0].flatten()
    for token in tokens:
        token = str(token)

        single_quoted = token.startswith("'") and token.endswith("'")
        double_quoted = token.startswith('"') and token.endswith('"')
        not_quoted = not single_quoted and not double_quoted

        is_date = DATE_REGEX.search(token) is not None
        is_digits = DIGITS_REGEX.search(token) is not None

        if not_quoted or is_date or is_digits:
            cleaned_tokens.append(token)
            continue

        sha = hashlib.sha256(SALT + token.encode('utf-8')).hexdigest()
        clean_token = "'{}\\{}'".format(len(token) - 2, sha)
        cleaned_tokens.append(clean_token)

    return ''.join(cleaned_tokens)

if ANONYMIZE:
    df['query_anon'] = df['query'].apply(anonymize)
    df['query_anon']

## Pre-processor: extracting query templates.

In [6]:
STRING_REGEX = r'([^\\])\'((\')|(.*?([^\\])\'))'
DOUBLE_QUOTE_STRING_REGEX = r'([^\\])"((")|(.*?([^\\])"))'
INT_REGEX = r'([^a-zA-Z])-?\d+(\.\d+)?'
HASH_REGEX = r'(\'\d+\\.*?\')'

def extract_template(query):
    template = query
    template = re.sub(HASH_REGEX, r"@@@", template)
    template = re.sub(STRING_REGEX, r"\1&&&", template)
    template = re.sub(DOUBLE_QUOTE_STRING_REGEX, r"\1&&&", template)
    template = re.sub(INT_REGEX, r"\1#", template)
    return template

query_column = 'query_anon' if ANONYMIZE else 'query'

df['query_template'] = df[query_column].apply(extract_template)
df['query_template']

0        SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...
1        SELECT I_PRICE, I_NAME , I_DATA   FROM item WH...
2        SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...
3        SELECT I_PRICE, I_NAME , I_DATA   FROM item WH...
4        SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...
                               ...                        
28029    SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...
28030    SELECT I_PRICE, I_NAME , I_DATA   FROM item WH...
28031    SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...
28032    SELECT I_PRICE, I_NAME , I_DATA   FROM item WH...
28033    SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...
Name: query_template, Length: 603434, dtype: object

In [7]:
df['log_time_s'] = df['log_time'].round('S')
df['log_time_s']

0       2021-12-06 16:01:21-05:00
1       2021-12-06 16:01:21-05:00
2       2021-12-06 16:01:21-05:00
3       2021-12-06 16:01:21-05:00
4       2021-12-06 16:01:21-05:00
                   ...           
28029   2021-12-06 16:01:54-05:00
28030   2021-12-06 16:01:54-05:00
28031   2021-12-06 16:01:54-05:00
28032   2021-12-06 16:01:54-05:00
28033   2021-12-06 16:01:54-05:00
Name: log_time_s, Length: 603434, dtype: datetime64[ns, pytz.FixedOffset(-300)]

In [8]:
gb = df.groupby(['query_template', 'log_time_s']).size()
grouped_df = pd.DataFrame(gb, columns=['count'])
grouped_df.drop('', axis=0, level=0, inplace=True)
grouped_df

count
query_template                                     log_time_s                      
DELETE FROM new_order WHERE NO_O_ID = #    AND ... 2021-12-06 16:01:12-05:00    110
                                                   2021-12-06 16:01:13-05:00    158
                                                   2021-12-06 16:01:14-05:00     75
                                                   2021-12-06 16:01:15-05:00    137
                                                   2021-12-06 16:01:16-05:00     90
...                                                                             ...
UPDATE warehouse   SET W_YTD = W_YTD + #  WHERE... 2021-12-06 16:02:08-05:00    159
                                                   2021-12-06 16:02:09-05:00    174
                                                   2021-12-06 16:02:10-05:00    161
                                                   2021-12-06 16:02:11-05:00    169
                                                   2021-12-06 16:02:12-05:00    126

[1929 rows x 1 columns]

## Clusterer

In [9]:
# TODO(WAN): Port online_clustering.py.
# TODO(WAN): I would be somewhat surprised if sklearn doesn't have this built in... We'll see

In [10]:
DF = grouped_df.copy()
assert DF.index.names == ['query_template', 'log_time_s']
assert DF.columns.values == ['count']


class Clusterer:
    def __init__(self, dataframe, n_samples=10000, rho=0.8):
        self._df = dataframe
        self.n_samples = n_samples
        self.rho = rho

        # Cluster every second.
        self.min_time = self._get_timestamps().min()
        self.max_time = self._get_timestamps().max()
        self.n = (self.max_time - self.min_time).days * 24*60*60 \
                 + (self.max_time - self.min_time).seconds \
                 + 1
        self.cluster_gap = 1
        self.n_gaps = self.n // self.cluster_gap + 1
    
        self._dbgname = {v:k for k,v in dict(enumerate(self._get_queries())).items()}
        
        
    def _get_queries(self):
        return sorted(set(self._df.index.get_level_values(0)))
    def _get_timestamps(self):
        return self._df.index.get_level_values(1)

    def _get_first_arrival(self, template):
        return self._df.xs(template, level=0).index.min()
    
    @staticmethod
    def _query_df_range(df, template, start_time, end_time):
        # The first level can be dropped since it is always just query_template == template.
        return df.query(
            "`query_template` == @template"
            " and @start_time <= `log_time_s`"
            " and `log_time_s` < @end_time"
        ).droplevel(0)
    
    @staticmethod
    def _query_df(df, template, timestamps):
        # The first level can be dropped since it is always just query_template == template.
        df = df.query(
            "`query_template` == @template"
            " and `log_time_s` in @timestamps"
        ).droplevel(0)
        return df.reindex(timestamps, fill_value=0)

    @staticmethod
    def _query_series(series, timestamps):
        series = series.query("`log_time_s` in @timestamps")
        return series.reindex(timestamps, fill_value=0)

    @staticmethod
    def _similarity(s1, s2):
        if s1.shape[0] == 0 or s2.shape[0] == 0:
            return 0
        # Reshape because we only have a single feature, the count.
        arr1 = s1.reshape(-1, 1)
        arr2 = s2.reshape(-1, 1)
        # Compute the cosine similarity.
        return sklearn.metrics.pairwise.cosine_similarity(arr1, arr2)[0][0]

    @staticmethod
    def _sample_timestamps(n, start_time, end_time, n_samples):
        if n > n_samples:
            offsets = np.random.choice(a=n, size=n_samples, replace=False)
        else:
            offsets = np.arange(n)
        timestamps = [start_time]
        for offset in offsets:
            next_time = pd.Timedelta(seconds=offset) + start_time
            if next_time >= end_time:
                break
            timestamps.append(next_time)
        return pd.array(timestamps)

    @staticmethod
    def _build_neighbors(centers, timestamps, n_neighbors):
        clusters = sorted(centers.keys())
        samples = np.array([
            Clusterer._query_series(centers[cluster], timestamps).values
            for cluster in clusters
        ])

        if len(samples) == 0:
            neighbors = None
        else:
            samples = samples.reshape(len(clusters), -1)
            normalized_samples = sklearn.preprocessing.normalize(samples, copy=False)
            neighbors = sklearn.neighbors.NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree', metric='l2')
            neighbors.fit(normalized_samples)
        return neighbors
        
    
    def cluster(self):
        rho = self.rho
        
        centers : Dict[int, pd.DataFrame] = {}
        cluster_totals : Dict[int, int] = {}
        cluster_sizes : Dict[int, int] = {}

        assignments = [
            (self.min_time, {template: None for template in sorted(self._get_queries())})
        ]
        
        current_time = self.min_time
        next_cluster = 0

        for gap in range(self.n_gaps):
            next_time = current_time + datetime.timedelta(seconds=self.cluster_gap)
            # Up to last 10 seconds.
            start_time = max(self.min_time, next_time - datetime.timedelta(seconds=10))
            timestamps = Clusterer._sample_timestamps(self.n, start_time, next_time, self.n_samples)

            last_assignment = assignments[-1][1]
            assignment = assignments[-1][1].copy()

            # Update counts for all the assignments made in the past round.
            for template in last_assignment:
                old_cluster = last_assignment[template]
                if old_cluster is not None:
                    counts = Clusterer._query_df_range(self._df, template, current_time, next_time)
                    centers[old_cluster] = centers[old_cluster].add(counts,
                                                                    fill_value=0)
                    cluster_totals[old_cluster] += counts.sum().values[0]

            # If possible, build a kdtree of neighbors.
            neighbors = Clusterer._build_neighbors(centers, timestamps, n_neighbors=1)
            
            # For each template, try to assign a cluster.
            for template in self._get_queries():
                old_cluster = assignment[template]
                
                if old_cluster is not None:
                    # Test if the template still belongs to its old cluster.
                    last_cluster_element = cluster_sizes[old_cluster] == 1
                    still_belongs = Clusterer._similarity(
                        Clusterer._query_df(self._df, template, timestamps).values,
                        Clusterer._query_series(centers[old_cluster], timestamps).values
                        ) > rho
                    # If the template still belongs, continue.
                    if last_cluster_element or still_belongs:
                        reason = ''
                        if last_cluster_element: reason += 'L'
                        if still_belongs: reason += 'B'
#                         print(f'Template stayed in cluster {old_cluster} because ({reason}): {self._dbgname[template]}')
                        continue
                    # Otherwise, eliminate the template from its old cluster.
                    cluster_sizes[old_cluster] -= 1
                    centers[old_cluster] = centers[old_cluster].sub(Clusterer._query_df_range(self._df, template, start_time, next_time),
                                                                    fill_value=0)
                    print(f'Template eliminated from cluster {old_cluster}: {self._dbgname[template]}')

                # Test if template has appeared at this point in time; otherwise, continue.
                if assignment[template] is None:
                    first_arrival = self._get_first_arrival(template)
                    if current_time <= first_arrival:
                        print(f'Template has not yet arrived at {current_time}, skipping: {self._dbgname[template]}')
                        continue
                    print(f'Template arrived at {current_time}: {self._dbgname[template]}')

                new_cluster = None
                # Try to assign to existing cluster.
                if neighbors == None:
                    for cluster in centers.keys():
                        if Clusterer._similarity(
                            self._query_df(self._df, template, timestamps).values,
                            self._query_series(centers[cluster], timestamps).values
                            ) > rho:
                            new_cluster = cluster
                            break
                else:
                    data = Clusterer._query_df(self._df, template, timestamps)['count'].values.reshape(1, -1)
                    data = sklearn.preprocessing.normalize(data)
                    neighbor = neighbors.kneighbors(data, return_distance=False)[0][0]
                    clusters = sorted(centers.keys())
                    if Clusterer._similarity(data, centers[clusters[neighbor]].values) > rho:
                        new_cluster = clusters[neighbor]

                # If this template found a cluster to join, then make the assignment and continue.
                if new_cluster is not None:
                    description = 'joined' if assignment[template] is None else 'reassigned to'
                    print(f'Template {description} cluster {new_cluster}: {self._dbgname[template]}')
                    assignment[template] = new_cluster
                    centers[new_cluster] = centers[new_cluster].add(
                        self._query_df_range(self._df, template, start_time, next_time),
                        fill_value=0)
                    cluster_sizes[new_cluster] += 1
                    continue

                # Otherwise, this template needs a new cluster. Make a new cluster.
                assignment[template] = next_cluster                
                centers[next_cluster] = self._query_df_range(self._df, template, start_time, next_time)
                assert centers[next_cluster].index.name == 'log_time_s'
                assert centers[next_cluster].columns.values == ['count']
                if centers[next_cluster].shape[0] == 0:
                    print(f'WARNING: cluster {cluster} has no items. Does the following query appear within the lookback window: {self._dbgname[template]}')

                cluster_sizes[next_cluster] = 1
                cluster_totals[next_cluster] = 0
                print(f'Created cluster {next_cluster} based on template: {self._dbgname[template]}')
                # Update the cluster counter.
                next_cluster += 1
        
            root = [None] * len(centers)
            # If possible, build an updated kdtree of neighbors.
            neighbors = Clusterer._build_neighbors(centers, timestamps, n_neighbors=2)

            clusters = sorted(centers.keys())
            if len(clusters) > 1:
                # Try to merge clusters.
                for i, cluster in enumerate(clusters):
                    merge_cluster = None
                    data = Clusterer._query_series(centers[cluster], timestamps)['count'].values.reshape(1, -1)
                    data = sklearn.preprocessing.normalize(data)
                    neighbor = neighbors.kneighbors(data, return_distance=False)

                    neighbor_inds = neighbor[0]
                    if clusters[neighbor_inds[0]] == cluster:
                        neighbor = neighbor_inds[1]
                    else:
                        neighbor = neighbor_inds[0]
                    while root[neighbor] is not None:
                        neighbor = root[neighbor]
                    is_similar = self._similarity(
                        self._query_series(centers[cluster], timestamps).values,
                        self._query_series(centers[clusters[neighbor]], timestamps).values) > rho
                    if cluster != clusters[neighbor] and is_similar:
                        merge_cluster = clusters[neighbor]
                    if merge_cluster != None:
                        centers[merge_cluster] = centers[merge_cluster].add(centers[cluster], fill_value=0)
                        cluster_sizes[merge_cluster] += cluster_sizes[cluster]
                        del centers[cluster]
                        del cluster_sizes[cluster]
                        if neighbors != None:
                            root[i] = neighbor
                        for template in self._get_queries():
                            if assignment[template] == cluster:
                                assignment[template] = merge_cluster
                                print(f'Template merged from cluster {cluster} into {merge_cluster}: {self._dbgname[template]}')
            assignments.append((next_time, assignment))
            current_time = next_time
            for cluster, df in centers.items():
                if df.shape[0] == 0:
                    print(f'WARNING: gap {gap} cluster {cluster} has no items.')
        for template, cluster in assignments[-1][1].items():
            print(self._dbgname[template], "->", cluster)
        
        self.assignments = assignments
        self.centers = centers
        self.cluster_totals = cluster_totals
        self.cluster_sizes = cluster_sizes
        self.num_clusters = len(self.centers)

    
clusterer = Clusterer(DF)
clusterer.cluster()

# (num clusters, assignment dict, cluster totals)

Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 0
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 1
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 2
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 3
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 4
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 5
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 6
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:01:11-05:00, skipping: 13
Te

Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 13
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 14
Template has not yet arrived at 2021-12-06 16:01:15-05:00, skipping: 35
Template has not yet arrived at 2021-12-06 16:01:16-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:16-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:16-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:16-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:16-05:00, skipping: 1

Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 13
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 14
Template has not yet arrived at 2021-12-06 16:01:23-05:00, skipping: 35
Template has not yet arrived at 2021-12-06 16:01:24-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:24-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:24-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:24-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:24-05:00, skipping: 1

Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 13
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 14
Template has not yet arrived at 2021-12-06 16:01:33-05:00, skipping: 35
Template has not yet arrived at 2021-12-06 16:01:34-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:34-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:34-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:34-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:34-05:00, skipping: 1

Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 13
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 14
Template has not yet arrived at 2021-12-06 16:01:43-05:00, skipping: 35
Template has not yet arrived at 2021-12-06 16:01:44-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:44-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:44-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:44-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:44-05:00, skipping: 1

Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 13
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 14
Template has not yet arrived at 2021-12-06 16:01:53-05:00, skipping: 35
Template has not yet arrived at 2021-12-06 16:01:54-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:01:54-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:01:54-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:01:54-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:01:54-05:00, skipping: 1

Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 11
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 12
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 13
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 14
Template has not yet arrived at 2021-12-06 16:02:03-05:00, skipping: 35
Template has not yet arrived at 2021-12-06 16:02:04-05:00, skipping: 7
Template has not yet arrived at 2021-12-06 16:02:04-05:00, skipping: 8
Template has not yet arrived at 2021-12-06 16:02:04-05:00, skipping: 9
Template has not yet arrived at 2021-12-06 16:02:04-05:00, skipping: 10
Template has not yet arrived at 2021-12-06 16:02:04-05:00, skipping: 1

Template arrived at 2021-12-06 16:02:13-05:00: 7
Created cluster 13 based on template: 7
Template arrived at 2021-12-06 16:02:13-05:00: 8
Created cluster 14 based on template: 8
Template arrived at 2021-12-06 16:02:13-05:00: 9
Created cluster 15 based on template: 9
Template arrived at 2021-12-06 16:02:13-05:00: 10
Created cluster 16 based on template: 10
Template arrived at 2021-12-06 16:02:13-05:00: 11
Created cluster 17 based on template: 11
Template arrived at 2021-12-06 16:02:13-05:00: 12
Created cluster 18 based on template: 12
Template arrived at 2021-12-06 16:02:13-05:00: 13
Created cluster 19 based on template: 13
Template arrived at 2021-12-06 16:02:13-05:00: 14
Created cluster 20 based on template: 14
Template arrived at 2021-12-06 16:02:13-05:00: 35
Created cluster 21 based on template: 35
0 -> 4
1 -> 4
2 -> 4
3 -> 4
4 -> 4
5 -> 8
6 -> 9
7 -> 13
8 -> 14
9 -> 15
10 -> 16
11 -> 17
12 -> 18
13 -> 19
14 -> 20
15 -> 4
16 -> 4
17 -> 4
18 -> 4
19 -> 4
20 -> 4
21 -> 4
22 -> 4
23 ->

In [99]:
# GenerateData()

query_cum = clusterer._df.groupby(level=0).cumsum()
MAX_CLUSTER_NUM=5
top_clusters = []
coverage_lists = [[] for i in range(MAX_CLUSTER_NUM)]

min_ts = clusterer._df.index.get_level_values(1).min()
last_ts = min_ts
online_clusters = {}

for current_ts, assignment in clusterer.assignments:
    cluster_totals = {}
    ts_total = 0

    for template, cluster in assignment.items():
        if cluster is None:
            continue

        query = query_cum.query("`query_template` == @template and `log_time_s` <= @current_ts")
        max_ts = query.index.get_level_values(1).max()
        
        template_total = 0
        if (current_ts - max_ts).seconds < 24*60*60:
            template_total = query.max().values[0]

        ts_total += template_total
        cluster_totals[cluster] = cluster_totals.get(cluster, 0) + template_total

    if len(cluster_totals) == 0:
        last_ts = current_ts
        continue
        
    sorted_clusters = sorted(cluster_totals.items(), key=lambda x:x[1], reverse=True)
    sorted_names, sorted_totals = zip(*sorted_clusters)
    
    lookahead = datetime.timedelta(seconds=10)
    
    current_top_clusters = sorted_clusters[:MAX_CLUSTER_NUM]
    for current_cluster, num_queries in current_top_clusters:
        if current_cluster not in online_clusters:
            online_clusters[current_cluster] = {}
            for template, cluster in assignment.items():
                if current_cluster != cluster:
                    continue
                start_ts = min_ts
                end_ts = last_ts + lookahead
                query = clusterer._df.query("`query_template` == @template and @start_ts <= `log_time_s` and `log_time_s` < @end_ts")
                online_clusters[cluster] = query
    
    current_top_cluster_names = [cluster for cluster, _ in current_top_clusters]
    for template, cluster in assignment.items():
        if cluster not in current_top_cluster_names:
            continue
        start_ts = last_ts + lookahead
        end_ts = current_ts + lookahead
        query = clusterer._df.query("`query_template` == @template and @start_ts <= `log_time_s` and `log_time_s` < @end_ts")
        online_clusters[cluster] = online_clusters[cluster].add(query, fill_value=0)
    
    top_clusters.append((current_ts, current_top_clusters))
    for i in range(MAX_CLUSTER_NUM):
        coverage_lists[i].append(sum(sorted_totals[:i+1] / ts_total))
    last_ts = current_ts

coverage = [sum(l) / len(l) for l in coverage_lists]
print(coverage)
for cluster in online_clusters:
    print(online_clusters[cluster])

[0.9843547386621334, 0.9877269682393068, 0.9897916265006498, 0.9918555797909601, 0.993894339235714]
                                                                              count
query_template                                     log_time_s                      
INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID... 2021-12-06 16:01:22-05:00  156.0
                                                   2021-12-06 16:01:23-05:00  148.0
                                                   2021-12-06 16:01:24-05:00  150.0
                                                   2021-12-06 16:01:25-05:00  150.0
                                                   2021-12-06 16:01:26-05:00  140.0
...                                                                             ...
UPDATE warehouse   SET W_YTD = W_YTD + #  WHERE... 2021-12-06 16:01:26-05:00  140.0
                                                   2021-12-06 16:01:27-05:00  173.0
                                                   2021-12-0

In [184]:
online_clusters.items()

dict_items([(0,                                                                               count
query_template                                     log_time_s                      
INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID... 2021-12-06 16:01:22-05:00  156.0
                                                   2021-12-06 16:01:23-05:00  148.0
                                                   2021-12-06 16:01:24-05:00  150.0
                                                   2021-12-06 16:01:25-05:00  150.0
                                                   2021-12-06 16:01:26-05:00  140.0
...                                                                             ...
UPDATE warehouse   SET W_YTD = W_YTD + #  WHERE... 2021-12-06 16:01:26-05:00  140.0
                                                   2021-12-06 16:01:27-05:00  173.0
                                                   2021-12-06 16:01:28-05:00  152.0
                                                   2021-12-0

In [170]:
trajs = {cluster: df.swaplevel().sum(level=0) for cluster, df in online_clusters.items()}
trajs

{0:                             count
 log_time_s                       
 2021-12-06 16:01:22-05:00  8340.0
 2021-12-06 16:01:23-05:00  7747.0
 2021-12-06 16:01:24-05:00  8387.0
 2021-12-06 16:01:25-05:00  7643.0
 2021-12-06 16:01:26-05:00  8459.0
 2021-12-06 16:01:27-05:00  8892.0
 2021-12-06 16:01:28-05:00  8861.0
 2021-12-06 16:01:29-05:00  8473.0
 2021-12-06 16:01:30-05:00  8485.0
 2021-12-06 16:01:11-05:00    17.0
 2021-12-06 16:01:12-05:00    86.0
 2021-12-06 16:01:13-05:00   110.0
 2021-12-06 16:01:14-05:00   130.0
 2021-12-06 16:01:15-05:00   123.0
 2021-12-06 16:01:16-05:00   141.0
 2021-12-06 16:01:17-05:00   117.0
 2021-12-06 16:01:18-05:00   129.0
 2021-12-06 16:01:19-05:00   117.0
 2021-12-06 16:01:20-05:00   137.0
 2021-12-06 16:01:21-05:00   136.0,
 1:                             count
 log_time_s                       
 2021-12-06 16:01:12-05:00   110.0
 2021-12-06 16:01:13-05:00   158.0
 2021-12-06 16:01:14-05:00    75.0
 2021-12-06 16:01:15-05:00   137.0
 2021-12-06 1

## Forecaster

In [145]:
# Remove first iteration of clustering?
traj = []
for ts, cluster_list in top_clusters[1:]:
    ts_list = [ts - datetime.timedelta(seconds=x) for x in range(10)]

    obs = []
    for cluster, _ in cluster_list:
        query = trajs[cluster].query("`log_time_s` < @ts")
        count = 0 if query.empty else query.iloc[-1]['count']
        obs.append(count)
    traj.append(np.array(obs))

def normalize(data):
    data_min = 1 - np.min(data)
    data = np.log(data + data_min)
    data_mean = np.mean(data)
    data -= data_mean
    data_std = np.std(data)
    data /= data_std
    return data, data_min, data_mean, data_std

normalize(traj)

(array([[ 1.19776016,  1.41033181,  1.41033181,  1.41033181,  1.41033181],
        [ 1.29582568,  0.97293276,  0.97293276,  0.96505306,  0.96505306],
        [ 1.26333359,  1.32661326,  1.32661326,  1.33088258,  1.33088258],
        [ 1.34350733,  1.07996072,  1.07996072,  1.07996072,  1.07996072],
        [ 1.23397604,  1.51859693,  1.51859693,  1.51859693,  1.51859693],
        [ 1.29129262,  1.07996072,  1.07996072,  1.07996072,  1.07996072],
        [ 1.23397604,  1.33932918,  1.33932918,  1.33932918,  1.33932918],
        [ 1.32661326,  1.29582568,  1.29582568,  1.29582568,  1.29582568],
        [ 1.4533043 ,  1.4533043 , -0.27172253, -0.27172253, -0.27172253],
        [ 1.33932918, -0.27172253, -0.27172253, -0.27172253, -1.56155614],
        [ 1.51859693, -0.27172253, -0.27172253, -0.27172253, -1.56155614],
        [ 1.33932918, -0.27172253, -0.27172253, -0.27172253, -1.56155614],
        [ 1.24883832, -0.27172253, -0.27172253, -0.27172253, -1.56155614],
        [ 1.37982996, -0.

In [12]:
# TODO(WAN): Port exp_multi_online_continuous.py
# Ok, most of the code is just wrapping around commandline args again.
# The actual model is just one fit() call to KR...

# Thursday
# 1. NP LSTM -- mike
# 2. Store input parameters and sample from those.
#    Won't be executing queries like course project so OK.
#    How to get them? -- wan

In [183]:
df

,log_time,command_tag,session_start_time,message,query,query_template,log_time_s
0,2021-12-06 16:01:21.493000-05:00,SELECT,2021-12-06 16:01:11-05:00,duration: 0.048 ms statement: SELECT S_QUANTI...,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_0...","SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...",2021-12-06 16:01:21-05:00
1,2021-12-06 16:01:21.493000-05:00,SELECT,2021-12-06 16:01:11-05:00,"duration: 0.033 ms statement: SELECT I_PRICE,...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...",2021-12-06 16:01:21-05:00
2,2021-12-06 16:01:21.494000-05:00,SELECT,2021-12-06 16:01:11-05:00,duration: 0.050 ms statement: SELECT S_QUANTI...,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_0...","SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...",2021-12-06 16:01:21-05:00
3,2021-12-06 16:01:21.494000-05:00,SELECT,2021-12-06 16:01:11-05:00,"duration: 0.033 ms statement: SELECT I_PRICE,...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...",2021-12-06 16:01:21-05:00
4,2021-12-06 16:01:21.494000-05:00,SELECT,2021-12-06 16:01:11-05:00,duration: 0.049 ms statement: SELECT S_QUANTI...,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_0...","SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...",2021-12-06 16:01:21-05:00
...,...,...,...,...,...,...,...
28029,2021-12-06 16:01:54.460000-05:00,SELECT,2021-12-06 16:01:11-05:00,duration: 0.053 ms statement: SELECT S_QUANTI...,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_0...","SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...",2021-12-06 16:01:54-05:00
28030,2021-12-06 16:01:54.460000-05:00,SELECT,2021-12-06 16:01:11-05:00,"duration: 0.034 ms statement: SELECT I_PRICE,...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...",2021-12-06 16:01:54-05:00
28031,2021-12-06 16:01:54.460000-05:00,SELECT,2021-12-06 16:01:11-05:00,duration: 0.050 ms statement: SELECT S_QUANTI...,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_0...","SELECT S_QUANTITY, S_DATA, S_DIST_#, S_DIST_#,...",2021-12-06 16:01:54-05:00
28032,2021-12-06 16:01:54.461000-05:00,SELECT,2021-12-06 16:01:11-05:00,"duration: 0.034 ms statement: SELECT I_PRICE,...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...","SELECT I_PRICE, I_NAME , I_DATA FROM item WH...",2021-12-06 16:01:54-05:00


In [13]:
# aggregate = 1 # agg by num minutes
# horizon = 60

# traj[cluster][time before date given] else 0


# kr train
# model.data = GeneratePair()
# evaluate_pass()


# trajs : cluster -> timestamp -> count

# csv files of form clusternum.csv
# time,count in each


# csv files containing
# num_queries, template
# time, count
# time, count
# ...
# where
# data[template][ts] = count
# data_cum[template][ts] = total